In [ ]:
import pandas as pd
import numpy as np

def convert(data):
    nodes = sorted(set([item for sublist in data for item in sublist]))
    node_map = {node: idx for idx, node in enumerate(nodes)}
    converted_data = [[node_map[item] for item in pair] for pair in data]
    return node_map, converted_data

def create_graph(data, n):
    graph = [[] for _ in range(n)]
    reverse_graph = [[] for _ in range(n)]
    for src, dest in data:
        graph[src].append(dest)
        reverse_graph[dest].append(src)
    return graph, reverse_graph

def dfs(graph, node, origin, visited, marker, results, path):
    for neighbor in graph[node]:
        if neighbor < origin:
            continue
        if marker[neighbor] == -2 and not visited[neighbor]:
            path.append(neighbor)
            if len(path) >= 2:
                results[len(path) - 2].append(path.copy())
            path.pop()
        if visited[neighbor] or (marker[neighbor] != origin and marker[neighbor] != -2):
            continue
        if len(path) == 4 or neighbor == origin:
            continue
        visited[neighbor] = True
        path.append(neighbor)
        dfs(graph, neighbor, origin, visited, marker, results, path)
        path.pop()
        visited[neighbor] = False

def mark_neighborhood(graph, node, origin, visited, marker, depth):
    for neighbor in graph[node]:
        if neighbor < origin or visited[neighbor]:
            continue
        marker[neighbor] = origin
        if depth < 3:
            visited[neighbor] = True
            mark_neighborhood(graph, neighbor, origin, visited, marker, depth + 1)
            visited[neighbor] = False

def find_cycles(df):
    unique_products = df['prod'].drop_duplicates().tolist()
    results_df = pd.DataFrame()
    group_id = 0
    
    for prod in unique_products:
        subset = df[df['prod'] == prod]
        data = subset[['buy_side_trdng_acnt_cd', 'sell_side_trdng_acnt_cd']].drop_duplicates().values.tolist()
        node_map, converted_data = convert(data)
        subset['buy_instn_cd'] = subset['buy_side_trdng_acnt_cd'].map(node_map)
        subset['sell_instn_cd'] = subset['sell_side_trdng_acnt_cd'].map(node_map)
        
        n = len(node_map)
        graph, reverse_graph = create_graph(converted_data, n)
        visited = [False] * n
        marker = [-1] * n
        path = []
        results = [[] for _ in range(4)]
        
        for i in range(n):
            mark_neighborhood(graph, i, i, visited, marker, 1)
            mark_neighborhood(reverse_graph, i, i, visited, marker, 1)
            
            for neighbor in reverse_graph[i]:
                marker[neighbor] = -2
            
            path.append(i)
            dfs(graph, i, i, visited, marker, results, path)
            path.pop()
            
            for neighbor in reverse_graph[i]:
                marker[neighbor] = i
        
        for cycle_set in results:
            for cycle in cycle_set:
                filtered_subset = subset[(subset['buy_instn_cd'].isin(cycle)) & (subset['sell_instn_cd'].isin(cycle))]
                if filtered_subset['yld_to_mrty'].nunique() == 1 and filtered_subset['nmnl_vol'].nunique() == 1:
                    filtered_subset['grp_id'] = group_id
                    results_df = pd.concat([results_df, filtered_subset])
                    group_id += 1
    
    return results_df

def extract_unique_codes(df):
    unique_codes = set()
    for codes in df['dl_cds'].dropna():
        unique_codes.update(codes.split(','))
    return list(unique_codes)

# 执行数据处理流程
df_res = find_cycles(df_all)
unique_codes = extract_unique_codes(df_res)
